# Reproducing MRIQC report

In [2]:
from pathlib import Path
from bids import BIDSLayout
from nibabel.freesurfer.mghformat import load
from brainspace.plotting import plot_hemispheres
from brainspace.mesh.mesh_io import read_surface
from nireports.assembler.report import Report
import numpy as np
import nibabel as nib

In [3]:
layout = BIDSLayout("bids_testset")

In [8]:
!ls bids_testset/sub-pp001/

ses-01


In [9]:
in_file = layout.get_file("sub-pp001/ses-01/anat/sub-pp001_ses-01_T1w.nii.gz")

In [10]:
entities = in_file.get_entities()
entities.pop("extension", None)
report_type = entities.pop("datatype", None)

In [11]:
print(entities)
print(report_type)

{'session': '01', 'subject': 'pp001', 'suffix': 'T1w'}
anat


In [12]:
bootstrap_file=f"bootstrap_data/bootstrap-{report_type}.yml"

In [8]:
prov = {}
prov["Input filename"] = f"Test filename"
prov["Version Freesurfer"] = f"Version X.Y"
prov["Execution environment"] = f"Test environment"

In [9]:
bids_meta = layout.get_file(in_file).get_metadata()

In [10]:
metadata={
    "dataset": "test name",
    "about-metadata": {
        "Provenance Information": prov,
        "Dataset Information": bids_meta,
    }
}

In [11]:
output_dir = "bids_testset/derivatives/testreport"
uuid = "test_uuid"
reportlets_dir = "bids_testset/derivatives/mriqc/sub-pp001/figures/"

In [12]:
robj = Report(output_dir, uuid, reportlets_dir=reportlets_dir,
              bootstrap_file=bootstrap_file, metadata=metadata,
              plugin_meta={}, **entities)

In [14]:
robj.generate_report()

0

# Apply nireports to Freesurfer output data

In [3]:
dir_FS ="bids_testset/derivatives/freesurfer/sub-pp001"

## Steps

1. Create figures as either .svg or .html in a `reportlets_dir`

Note: each figure is a "reportlet"

2. Create a bootstrap_file which will gather the appropriate reportlets in sections

3. Create dictionaries for each set of textual information (e.g. brain region volumes)

Note: each set title `set_title` and asscociated dictionary `dict` are to be the values of `metadata["about-metadata"]` so that `metadata["about-metadata"]["set_title'] = dict`

4. Generate the report by generating an instance of `Report` and calling the `generate_report` method

## Step 1: creating figures

In [4]:
# Load native pial surface
pial_lh = read_surface(dir_FS+'/surf/lh.pial', itype='fs')
pial_rh = read_surface(dir_FS+'/surf/rh.pial', itype='fs')

# Load native mid surface
mid_lh = read_surface(dir_FS+'/surf/lh.midthickness', itype='fs')
mid_rh = read_surface(dir_FS+'/surf/rh.midthickness', itype='fs')

# Load native white matter surface
wm_lh = read_surface(dir_FS+'/surf/lh.white', itype='fs')
wm_rh = read_surface(dir_FS+'/surf/rh.white', itype='fs')

# Load native inflated surface
inf_lh = read_surface(dir_FS+'/surf/lh.inflated', itype='fs')
inf_rh = read_surface(dir_FS+'/surf/rh.inflated', itype='fs')

### Thickness: Inflated native surface

In [5]:
th_lh = dir_FS + '/surf/lh.thickness'
th_rh = dir_FS + '/surf/rh.thickness'
th_nat = np.hstack(np.concatenate((nib.freesurfer.read_morph_data(th_lh),
                                   nib.freesurfer.read_morph_data(th_rh)), axis=0))

In [ ]:
# Plot the surface
plot_hemispheres(inf_lh, inf_rh, array_name=th_nat, size=(900, 250), color_bar='bottom', zoom=1.25, embed_nb=True, 
                 interactive=False, share='both', nan_color=(0, 0, 0, 1), color_range=(1.5, 4), cmap="inferno", 
                 transparent_bg=False)

In [8]:
th_nat.shape

(276355,)

In [13]:
inf_lh.n_points * 2

275684

In [ ]:
from functions import generate_volume_dictionary, generate_paths, generate_table_volume


aseg_paths = generate_paths("bids_testset")
table = generate_table_volume(aseg_paths)

dictionary = generate_volume_dictionary(table)
print(dictionary)

layout = BIDSLayout("bids_testset")

in_file = layout.get_file("sub-pp001/ses-01/anat/sub-pp001_ses-01_T1w.nii.gz")

entities = in_file.get_entities()
entities.pop("extension", None)
report_type = entities.pop("datatype", None)

bids_meta = layout.get_file(in_file).get_metadata()

bootstrap_file=f"bootstrap_data/bootstrap-{report_type}.yml"

prov = {}
prov["Input filename"] = f"Test filename"
prov["Version Freesurfer"] = f"Version X.Y"
prov["Execution environment"] = f"Test environment"

metadata={
    "dataset": "test name",
     "about-metadata": {
         "Provenance Information": prov,
         "Volume metrics": dictionary,
         "Dataset Information": bids_meta
         }
}

output_dir = "bids_testset/derivatives/testreport"
uuid = "test_uuid"
reportlets_dir = "bids_testset/derivatives/mriqc/sub-pp001/figures/"

robj = Report(output_dir, uuid, reportlets_dir=reportlets_dir,
              bootstrap_file=bootstrap_file, metadata=metadata,
              plugin_meta={}, **entities)

robj.generate_report()
